In [1]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import time
import json

In [2]:
# маппинг регионов и кодов

In [3]:
data = pd.read_csv('../data/vacancies_ru.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head(10)
# area_regionId -- код города, надо смотреть его
# area_id -- ненужное

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder
0,v_862116,Смотритель музейный,c_162972,"['Пользователь ПК', 'Работа в команде', 'Умени...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Смотритель музейный,32,4181,69972,5,5,1
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"['Активные продажи', 'Холодные продажи', 'Кред...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Ведущий менеджер по работе с физическими лицами,17,6731,120750,5,5,1
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,NaN,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,Бухгалтер,81,5806,109013,5,5,3
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,NaN,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience,Пекарь,52,4218,235992,5,5,1
4,v_312507,Торговый представитель (г. Абакан),c_206699,"['Продуктивность', 'Клиентоориентированность',...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Торговый представитель,39,6487,118557,5,5,2
5,v_2323706,Менеджер по качеству,c_244141,"['Контроль качества', 'Производственный контро...",88000.0,NaN,RUR,a_5920,ar_64,full,fullDay,between1And3,Менеджер по качеству,66,5469,160161,5,5,2
6,v_704266,Менеджер по продажам / Sales manager,c_192038,"['Активные продажи', 'Поиск и привлечение клие...",30000.0,100000.0,RUR,a_5398,ar_71,full,fullDay,between1And3,Менеджер по продажам,74,4888,102268,5,5,2
7,v_2091086,Монтажник СТиЖБК,c_230890,NaN,100000.0,NaN,RUR,a_500,ar_36,full,flyInFlyOut,between1And3,Монтажник СТиЖБК,35,4447,145437,5,1,2
8,v_839657,Подсобный рабочий (строительный объект),c_192370,"['Умение работать в команде', 'Уборка строител...",50000.0,50000.0,RUR,a_2649,ar_44,full,fullDay,noExperience,Подсобный рабочий,44,1834,102637,5,5,1
9,v_2485961,Домработница/Домработник,c_133807,"['Уборка, стирка, домработница, помощница по х...",65000.0,75000.0,RUR,a_1756,ar_41,full,fullDay,moreThan6,Домработница,41,842,37567,5,5,4


In [4]:
data['area_regionId'].unique()

array(['ar_33', 'ar_2', 'ar_78', 'ar_51', 'ar_4', 'ar_64', 'ar_71',
       'ar_36', 'ar_44', 'ar_41', 'ar_87', 'ar_1', 'ar_23', 'ar_39',
       'ar_72', 'ar_20', 'ar_37', 'ar_94', 'ar_21', 'ar_69', 'ar_99',
       'ar_38', 'ar_91', 'ar_10', 'ar_25', 'ar_59', 'ar_97', 'ar_81',
       'ar_52', 'ar_49', 'ar_28', 'ar_16', 'ar_77', 'ar_56', 'ar_45',
       'ar_70', 'ar_66', 'ar_8', 'ar_48', 'ar_103', 'ar_42', 'ar_96',
       'ar_11', 'ar_31', 'ar_65', 'ar_79', 'ar_102', 'ar_13', 'ar_92',
       'ar_101', 'ar_22', 'ar_89', 'ar_55', 'ar_0', 'ar_47', 'ar_76',
       'ar_7', 'ar_61', 'ar_104', 'ar_84', 'ar_74', 'ar_40', 'ar_3',
       'ar_73', 'ar_32', 'ar_100', 'ar_68', 'ar_58', 'ar_5', 'ar_82',
       'ar_18', 'ar_6', 'ar_95', 'ar_9', 'ar_54', nan, 'ar_90', 'ar_60',
       'ar_63', 'ar_67', 'ar_15', 'ar_85', 'ar_12', 'ar_17', 'ar_19',
       'ar_24', 'ar_88', 'ar_26', 'ar_62', 'ar_93', 'ar_46', 'ar_14',
       'ar_35', 'ar_83', 'ar_30', 'ar_43'], dtype=object)

In [5]:
data['area_regionId'].value_counts()

ar_41    400518
ar_78    178072
ar_64    151976
ar_21     90867
ar_44     88790
          ...  
ar_46        27
ar_35        25
ar_30        19
ar_14        14
ar_43         1
Name: area_regionId, Length: 95, dtype: int64

In [6]:
# дерево регионов
url = f'https://api.hh.ru/areas'

response = requests.get(url)

if response.status_code == 200:
    tree = response.json()
    
tree[8]

{'id': '97',
 'parent_id': None,
 'name': 'Узбекистан',
 'areas': [{'id': '2763', 'parent_id': '97', 'name': 'Аккурган', 'areas': []},
  {'id': '2764', 'parent_id': '97', 'name': 'Акташ (Узбекистан)', 'areas': []},
  {'id': '2765', 'parent_id': '97', 'name': 'Алат (Узбекистан)', 'areas': []},
  {'id': '2766', 'parent_id': '97', 'name': 'Алмалык', 'areas': []},
  {'id': '2767', 'parent_id': '97', 'name': 'Ангрен', 'areas': []},
  {'id': '2768', 'parent_id': '97', 'name': 'Андижан', 'areas': []},
  {'id': '2769', 'parent_id': '97', 'name': 'Асака', 'areas': []},
  {'id': '2770', 'parent_id': '97', 'name': 'Ахангаран', 'areas': []},
  {'id': '2771', 'parent_id': '97', 'name': 'Ахунбабаев', 'areas': []},
  {'id': '2772', 'parent_id': '97', 'name': 'Байсун', 'areas': []},
  {'id': '2773', 'parent_id': '97', 'name': 'Бахт', 'areas': []},
  {'id': '2774', 'parent_id': '97', 'name': 'Бекабад', 'areas': []},
  {'id': '2775', 'parent_id': '97', 'name': 'Беруни', 'areas': []},
  {'id': '2776', 'p

In [7]:
len(tree)

9

In [8]:
tree[0]['areas'][0]

{'id': '1620',
 'parent_id': '113',
 'name': 'Республика Марий Эл',
 'areas': [{'id': '4228',
   'parent_id': '1620',
   'name': 'Виловатово',
   'areas': []},
  {'id': '1621', 'parent_id': '1620', 'name': 'Волжск', 'areas': []},
  {'id': '1622', 'parent_id': '1620', 'name': 'Звенигово', 'areas': []},
  {'id': '4229', 'parent_id': '1620', 'name': 'Знаменский', 'areas': []},
  {'id': '61', 'parent_id': '1620', 'name': 'Йошкар-Ола', 'areas': []},
  {'id': '4230', 'parent_id': '1620', 'name': 'Кельмаксола', 'areas': []},
  {'id': '4231', 'parent_id': '1620', 'name': 'Килемары', 'areas': []},
  {'id': '1623', 'parent_id': '1620', 'name': 'Козьмодемьянск', 'areas': []},
  {'id': '4232',
   'parent_id': '1620',
   'name': 'Красногорский (Республика Марий Эл)',
   'areas': []},
  {'id': '4233',
   'parent_id': '1620',
   'name': 'Краснооктябрьский',
   'areas': []},
  {'id': '7178',
   'parent_id': '1620',
   'name': 'Красный Стекловар',
   'areas': []},
  {'id': '4234', 'parent_id': '1620', 

In [9]:
# маппинг id городов и названий
def id_region_mapping(branch):
    id_to_names = {}
    if not branch['areas']:
        id_to_names[branch['id']] = branch['name']
    else:
        for subbranch in branch['areas']:
            id_to_names.update(id_region_mapping(subbranch))

    return id_to_names

In [10]:
region_id_to_names = {tree[0]['id']: tree[0]['name']} 

for branch in tree:
    print(f"{branch['id']} -- {branch['name']}")
    region_id_to_names.update(id_region_mapping(branch))

113 -- Россия
5 -- Украина
40 -- Казахстан
9 -- Азербайджан
16 -- Беларусь
28 -- Грузия
1001 -- Другие регионы
48 -- Кыргызстан
97 -- Узбекистан


In [11]:
region_id_to_names['1']

'Москва'

In [21]:
region_id_to_names['2']

'Санкт-Петербург'

In [14]:
print(len(region_id_to_names.keys()))

9445


In [15]:
with open('region_id_to_names.json', 'w') as file:
    json.dump(region_id_to_names, file)

In [16]:
names_to_region_id = {}

for key, value in region_id_to_names.items():
    names_to_region_id[value] = key
    
with open('names_to_region_id.json', 'w') as file:
    json.dump(names_to_region_id, file)

In [19]:
names_to_region_id['Москва']

'1'

In [20]:
names_to_region_id['Россия']

'113'

In [18]:
# получение резюме вида hh для датасета

In [22]:
url = 'https://hh.ru/search/resume?text=&area=113&isDefaultArea=true&exp_period=all_time&logic=normal&pos=full_text&hhtmFrom=vacancy_search_list&hhtmFromLabel=resume_search_line'

response = requests.get(url)

if response.status_code == 200:
    tree = response.json()
tree

[{'id': '113',
  'parent_id': None,
  'name': 'Россия',
  'areas': [{'id': '1620',
    'parent_id': '113',
    'name': 'Республика Марий Эл',
    'areas': [{'id': '4228',
      'parent_id': '1620',
      'name': 'Виловатово',
      'areas': []},
     {'id': '1621', 'parent_id': '1620', 'name': 'Волжск', 'areas': []},
     {'id': '1622', 'parent_id': '1620', 'name': 'Звенигово', 'areas': []},
     {'id': '4229', 'parent_id': '1620', 'name': 'Знаменский', 'areas': []},
     {'id': '61', 'parent_id': '1620', 'name': 'Йошкар-Ола', 'areas': []},
     {'id': '4230', 'parent_id': '1620', 'name': 'Кельмаксола', 'areas': []},
     {'id': '4231', 'parent_id': '1620', 'name': 'Килемары', 'areas': []},
     {'id': '1623',
      'parent_id': '1620',
      'name': 'Козьмодемьянск',
      'areas': []},
     {'id': '4232',
      'parent_id': '1620',
      'name': 'Красногорский (Республика Марий Эл)',
      'areas': []},
     {'id': '4233',
      'parent_id': '1620',
      'name': 'Краснооктябрьский',